In [1]:
import torch
import os
import re
import json
from copy import deepcopy
from datasets import load_dataset
os.environ["HF_Home"] = "cache"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
with open(".env") as f:
    token = f.read().strip()
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
quantization_config = BitsAndBytesConfig(load_in_8bit=False)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=token, use_fast=True, cache_dir="cache")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=token, cache_dir="cache", torch_dtype=torch.bfloat16, device_map="cuda:0", quantization_config=quantization_config)
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", load_in_4bit=True, token=token, cache_dir="cache", device_map="auto")

/users/gmongaras/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-10 08:00:41.328607: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 08:00:42.127735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.33s/it]


In [17]:
policy = torch.load("models/policy_1200.pt", weights_only=False).cuda()

In [18]:
batch_size = 2
group_batch_size = 4
num_steps = 1000
max_length = 512

In [19]:
tokenizer.model_max_length = max_length
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Generate a  
question = "Maya packed a gift box for her cousin. She first placed the empty box on a scale and added enough cookies to make the box weigh 3 pounds. Then, she added enough chocolate bars to double the weight. After that, she added another 3 pounds of cookies. Finally, she added enough candy canes to triple the weight of the box at that point. What was the final weight of the box, in pounds? Translate the answer to binary. then translate it to hex from binary." 
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You will be given a question. First reason about the question and answer. Then answer the question.
The final solution must be in json format:
{
    "solution": "put your solution here"
}
Your solution should only include the final answer in the format above.<|eot_id|><|start_header_id|>user<|end_header_id|>

""" + f"Question: {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
max_length = 512
temperature = 0.7
inputs = tokenizer(prompt, return_tensors="pt", padding_side="left")
# Generate text
outputs = policy.generate(input_ids=inputs["input_ids"].cuda(), attention_mask=inputs["attention_mask"].cuda(), max_length=max_length, do_sample=True, temperature=temperature)
out = tokenizer.decode(outputs[0], skip_special_tokens=True)
search_string = """
{\s*"solution": .*\s*}
""".strip()
ans = json.loads(re.findall(search_string, out)[-1])["solution"]
print(f"\n\n\nAnswer: {ans}\n\n\n")
print(f"Full output-------------------\n{out}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.





Answer: 1B



Full output-------------------
system

You will be given a question. First reason about the question and answer. Then answer the question.
The final solution must be in json format:
{
    "solution": "put your solution here"
}
Your solution should only include the final answer in the format above.user

Question: Maya packed a gift box for her cousin. She first placed the empty box on a scale and added enough cookies to make the box weigh 3 pounds. Then, she added enough chocolate bars to double the weight. After that, she added another 3 pounds of cookies. Finally, she added enough candy canes to triple the weight of the box at that point. What was the final weight of the box, in pounds? Translate the answer to binary. then translate it to hex from binary.assistant

Step 1: Analyze the problem

Maya starts with a 0-pound box and adds weights incrementally.

Step 2: Determine the weight after each addition

1. Initial weight = 0
2. First addition: Cookies add 3 pounds t